In [1]:
import polars as pl

In [2]:
restaurantes = pl.read_csv('/content/drive/MyDrive/Alura/restaurantes.csv', encoding='utf8')

In [3]:
restaurantes

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,str,i64,str,str,str,str,str,str,str,str
"""Jalsa""","""Yes""","""Yes""","""4.1/5""",775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""","""800""","""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""","""4.1/5""",787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""","""800""","""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""","""3.8/5""",918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""","""800""","""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""","""3.7/5""",88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""","""300""","""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""","""3.8/5""",166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""","""600""","""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""","""3.6 /5""",27,"""Bar""",null,"""Continental""","""1,500""","""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""","""600""","""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""","""2,000""","""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [4]:
restaurantes.glimpse()

Rows: 51717
Columns: 13
$ name                        <str> 'Jalsa', 'Spice Elephant', 'San Churro Cafe', 'Addhuri Udupi Bhojana', 'Grand Village', 'Timepass Dinner', 'Rosewood International Hotel - Bar & Restaurant', 'Onesta', 'Penthouse Cafe', 'Smacznego'
$ online_order                <str> 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes'
$ book_table                  <str> 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No'
$ rate                        <str> '4.1/5', '4.1/5', '3.8/5', '3.7/5', '3.8/5', '3.8/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5'
$ votes                       <i64> 775, 787, 918, 88, 166, 286, 8, 2556, 324, 504
$ rest_type                   <str> 'Casual Dining', 'Casual Dining', 'Cafe, Casual Dining', 'Quick Bites', 'Casual Dining', 'Casual Dining', 'Casual Dining', 'Casual Dining, Cafe', 'Cafe', 'Cafe'
$ dish_liked                  <str> 'Pasta, Lunch Buffet, Masala Papad, Paneer Lajawab, Tomato Shorba, Dum Biryani, Sweet Corn Soup', 'Mom